Note: Below is only for managed Identity:

In [1]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="XXX")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="XXX"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

Install libraries:

In [103]:
#pip install python-dotenv

In [1]:
#pip install openai

In [50]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-01-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [1]:
#print(os.getenv("AZURE_OPENAI_ENDPOINT"))

## 1. Understand the AOAI Models' capabilities. Start with the latest model, prove your idea , then test with smaller models. 
Model size is critical for better performance.

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

** Length ** control (specify desired output length e.g.: number of words)

** Tone ** control (e.g.: polite, passionate, professional, technical, funny, casual, serious etc.)

** Style ** control (e.g.: in the style of Shakespeare, JK Rowling, Nelson Mandela etc.)

** Audience ** control (e.g.: a 5-year-old can understand etc)

** Context ** control (e.g.: news, novel, textbook, report, white paper, blog etc.)

In [54]:
base_system_message = "You are a helpful assistant."
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = " Write a 2 paragraph inspiring poem focussing on products of Howden Group Holdings in a funny way."
# Write a 2 paragraph inspiring poem about Howden Group Holdings
# Write a 2 paragraph inspiring poem focussing on products of Howden Group Holdings in a funny way
# Write a 2 paragraph inspiring poem focussing on products of Howden Group Holdings in the style of Shakespeare

In [ ]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [6]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [7]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    max_tokens=60
)

# Print the result
print(response.choices[0].message.content.strip())


## 4. Articulate the desired output format through examples

In [56]:
system_message = "You are a helpful assistant."

In [57]:
user_message=f"""Extract the entities mentioned in the text below. 
First extract all company names, then extract all years, 
then extract specific topics which fit the content and finally extract general overarching themes\n\n 
Desired format: 
Company names: <comma_separated_list_of_company_names> 
Years: 
Specific topics:
General themes: 
### Text:
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n\n 
###
"""


In [ ]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

## 5.Start with zero-shot, then few-shot (example)

In [117]:
system_message = "You are a helpful assistant."

In [118]:
prompt_zero=f"""Extract most important keywords from the corresponding texts below.\n\n 

###Text: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords:###
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_zero}
    ]
)

print(response.choices[0].message.content)

In [120]:
prompt_few=f"""Extract most important keywords from the corresponding texts below.\n\n 
### Text 1: 
Stripe provides APIs that web developers can use to integrate 
payment processing into their websites and mobile applications. \n
Keywords 1: Stripe, payment processing, APIs, web developers, websites 
### 

###Text 2: 
OpenAI has trained cutting-edge language models that are very good at understanding 
and generating text. Our API provides access to these models and can be used to solve virtually 
any task that involves processing language. \n
Keywords 2: OpenAI, language models, text processing, API.
### 

###Text 3: 
We’re happy to announce that OpenAI and Microsoft are extending our partnership.
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments 
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is 
increasingly safe, useful, and powerful. \n
Keywords 3:"""

In [ ]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_few}
    ]
)

print(response.choices[0].message.content)

## 6.Instead of just saying what not to do, say what to do instead

In [59]:
system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. 
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

## 7. Divide complex tasks into sub-tasks

In [64]:
system_message = "You are a helpful assistant."

In [66]:
text = f"""
As an FSI company, we want our pension schemes to have a positive impact on our customers. Whether you're just starting to save into a pension or ready to take money out of it, we have the best interests of our members and customers at heart. It's about long-term financial wellbeing and sharing responsibility for building a better future. Save today to enjoy tomorrow."""
# example 1
user_message = f"""
Perform the actions below by separating your answers with line breaks. 
1 - Summarize the following text below with 1 sentence in English.
2 - Translate the summary into Hindi.
3 - List each company name in the Hindi summary.
4 - Output a json object that contains the following:
keys: Hindi_summary, Hindi_company_names.


###
Text:
{text} 
###
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

print(response.choices[0].message.content)

## 8. Chain of Thought

The language model is prompted to generate a few intermediate reasoning steps to arrive at the final answer. 

Uses "greedy decoding" which means selecting the most likely token (word or character) at each step of the sequence generation process. At each time step, the model predicts the next token based on the previously generated tokens, and the token with the highest predicted probability is chosen as the output for that step. This process is repeated until the desired sequence length is reached or until a special end-of-sequence token is generated.

**Temp=0** is used because it uses greedy decoding. It first creates the greedy coding and then the answer.

In [71]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": PROMPT_ZERO_SHOT}
    ],
    temperature=0,
    max_tokens=200,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


In [73]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": PROMPT_ZERO_SHOT_CoT}
    ],
    temperature=0,
    max_tokens=300,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


In [29]:
prompt_fsi_zeroshot_cot= """Q: Let’s analyze the financial health of Company X. The company has a revenue of $10 million, expenses of $7 million, and a debt of $2 million. Calculate the net profit and the debt-to-equity ratio.
A: The answer is"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_fsi_zeroshot_cot}
    ],
    temperature=0,
    max_tokens=100,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


In [31]:
prompt_fsi_zeroshot_cot= """Q: Let’s analyze the financial health of Company X. The company has a revenue of $10 million, expenses of $7 million, and a debt of $2 million. Calculate the net profit and the debt-to-equity ratio.
A: Let's think step by step."""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_fsi_zeroshot_cot}
    ],
    temperature=0,
    max_tokens=1000,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


In [33]:
PROMPT_FEW_SHOT_CoT = """
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A: Elif had £10 at the beginning. When she consumed £2, 10-2=8 , £8 remains.
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A:
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": PROMPT_FEW_SHOT_CoT}
    ],
    temperature=0,
    max_tokens=1000,
    stop=["\nQ:"]
)

# Print the result
print(response.choices[0].message.content.strip())


** Auto-COT ** uses zero-shot-cot results just like few-shot learning for reasoning. Instead of using few-shot-cot, auto-cot can be useful and easy because you don't need to create manual examples (labels/reasonings)

In [77]:
prompt_auto_cot = """
Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to his friend. How many tennis balls does
he have now?
A: Lets think step by step.Roger had 5 tennis balls at the beginning. He bought 2 cans of tennis balls, each with 3 balls, so he now has 5+2x3=11 tennis balls. After giving 4 to his friend, he has 11-4=7 tennis balls remaining.
Q: Elif went to market with £10 and consumed £2. How much does she have now?
A:
"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_auto_cot}
    ],
    temperature=0.5,
    max_tokens=1000
)

# Access the message content safely
message_content = response.choices[0].message.content
if message_content:
    print(message_content.strip())
else:
    print("The response content is empty.")


## 9. Self Consistency

Self-consistency aims "to replace the naive greedy decoding used in chain-of-thought prompting". The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to ** select the most consistent answer.**

In the chat scenarios, **Asking the model to self-verify** its own responses. Like a student double-checking their answers, the AI model cross-references its responses to maintain consistency. 

In [39]:
prompt= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister?"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.5,
    max_tokens=230,
    stop=["\nA:"]
)

# Print the result
print(response.choices[0].message.content.strip())


In [45]:
prompt2= f"""When I was 6, my sister was half my age. Now
I am 70 how old is my sister? Let's think step by step"""

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  # Ensure this is the correct model identifier
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt2}
    ],
    temperature=0,
    max_tokens=1000
)

# Print the result
print(response.choices[0].message.content.strip())


In [47]:
prompt3=f"""
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

In [48]:
from openai import OpenAI

def call_openai(num_times, start_phrase, temperature):
    # Initialize the OpenAI client
    #client = OpenAI(api_key='your-api-key')  # Replace with your actual API key

    for i in range(num_times):
        # Define the messages for the chat completion
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": start_phrase}
        ]

        # Send a chat completion request
        response = client.chat.completions.create(
            model="gpt-4o",  # Use the appropriate model name
            messages=messages,
            temperature=temperature,
            max_tokens=100
        )

        # Extract and print the assistant's reply
        reply = response.choices[0].message.content.strip()
        print(f"Response {i + 1}:\n{reply}")
        print("*****************************")


In [ ]:
call_openai(10, prompt3, temperature = 1)

# 9. Step-Back Prompting Technique

You start by providing the model with a prompt or question.
The model generates a response based on the initial prompt.
Instead of immediately accepting the response, you prompt the model to review or analyse its own response. This could involve asking the model to check for errors, verify facts, or consider alternative approaches.
Based on the reassessment, the model generates a refined or corrected response.

In [9]:
# function to generate a response based on a prompt  
def generate_response(prompt):  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": prompt}  
    ]  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=messages  
    )  
    return response.choices[0].message.content.strip()  

# function to rephrase the input text  
def rephrase_input(input_text):  
    rephrase_prompt = (  
        "You are a helpful assistant. Please rephrase the following request to make it clearer and a more general question: "  
        f"'{input_text}'"  
    )  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": rephrase_prompt}  
    ]  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=messages  
    )  
    return response.choices[0].message.content.strip()  
  
  
# function to reword and check the answer  
def reword_and_check_answer(original_answer):  
    rewording_prompt = f"Reword and verify the following statement: '{original_answer}'"  
    messages = [  
        {"role": "system", "content": "You are a helpful assistant."},  
        {"role": "user", "content": rewording_prompt}  
    ]  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=messages  
    )  
    return response.choices[0].message.content.strip()

In [ ]:
initial_prompt = "Howden Group Holdings, a global insurance group with employee at their heart and a focus on innovation, is committed to providing the best insurance solutions for its clients."  
  
# Generate a response based on the initial prompt  
response = generate_response(initial_prompt)  
print("*** Initial Response:\n", response)  
  
# Reword and verify the answer  
checked_response = reword_and_check_answer(response)  
print("*** Reworded and Checked Answer:\n", checked_response)  
  
print("*** Final Response:\n", checked_response)  
  
# if __name__ == "__main__":  
#     initial_prompt = input("Enter your question: ")  
#     response = generate_response(initial_prompt)  
#     print("Initial Response:\n", response)  
#     checked_response = reword_and_check_answer(response)  
#     print("Reworded and Checked Answer:\n", checked_response)  

## 10. Iterative approach

Prompt engineering is an iterative process. If you're unsatisfied with the AI's response, refine your prompt and try again. Analyze the results you receive and consider adjusting your prompt's context, clarity, or structure. This process of trial and error will help you better understand how the AI model interprets your prompts and allow you to fine-tune your approach.

·        Try different prompts to find what works best

·        When attempting few-shot learning, try also to include direct instructions

·        Rephrase a direct instruction set to be more or less concise, e.g.: taking a previous example and giving the next instruction without having to repeat the input

·        Try different personas keywords to see how it affects the response style

·        Use fewer or more examples in the few-shot learning

·        Co-create with AI: An example of a very useful prompt to get a good output from the LLM :

In [95]:
text=f"""
Overall Results
Year Ended December 31, 2023 versus 2022:
Net income attributable to  ordinary shareholders was $1.1 billion for the year ended December 31, 2023, 
which compares to a net loss of $906 million from 2022, as a result of:
• Favorable total investment returns recognized in net income of $1.1 billion for the year ended December 31, 
2023, consisting of the aggregate of net investment income, net realized (losses) gains, net unrealized gains 
(losses) and income (losses) from equity method investments, in comparison to negative total investment 
returns included in net income of $1.2 billion for the year ended December 31, 2022. The variance in total 
investment returns recognized in net income was driven by: 
◦ Net unrealized gains on our other investments, including equities of $397 million, in comparison to net 
unrealized losses in 2022 of $433 million, as a result of strong global equity market performance, 
particularly in the first and fourth quarters of 2023, and tightening high yield credit spreads, in comparison to 
the challenging market environment for the year ended December 31, 2022;
◦ Net realized and unrealized gains on our fixed maturities of $66 million in 2023, compared to net realized 
and unrealized losses of $1.2 billion in 2022, primarily due to a decrease in interest rates across U.S., U.K. 
and European markets in 2023 as compared to significant increases in interest rates in 2022; 
◦ An increase in net investment income of $192 million in 2023 when compared to 2022, consistent with the 
increasing investment income we have earned on a sequential quarterly basis, primarily due to the 
reinvestment of fixed maturities at higher yields, deployment of consideration received from LPT and 
insurance transactions closed over the past 12 months and the impact of rising interest rates on our fixed 
maturities securities that are subject to floating interest rates; and
◦ Income from equity method investments of $13 million, driven by income from our investments in Core 
Specialty and Citco, partially offset by losses from our investment in Monument Re, compared to losses of 
$74 million in 2022, primarily driven by losses from our investment in Monument Re. 
• An increase in other income of $241 million in 2023 when compared to 2022, largely driven by the first quarter 
2023 net gain recognized from the novation of the Enhanzed Re reinsurance of a closed block of life annuity 
policies; and 
• A favorable change in income tax benefit of $238 million, primarily driven by the establishment of a $205 million 
net deferred tax asset related to the enactment of the Bermuda Corporate Income Tax in December 2023. We 
also recorded a $25 million partial release of our deferred tax asset valuation allowance as a result of increases 
in projected taxable income in the U.S. and a reduction in deferred tax assets associated with decreases in 
unrealized losses on investment securities reported in AOCI in the U.S. and U.K. jurisdictions. This was partially 
offset by an increase in the valuation allowance in our U.K. and EU jurisdictions primarily due to losses, 
whereby no corresponding tax benefits were recognized for the period. 
"""

In [96]:
prompt_iterative= f""" Your task is to explain given information in a very simple way.
### Context:
{text}
"""

In [ ]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative}
    ]
)

print(response.choices[0].message.content)

**Issue 1:** I want to keep numerical values in more readable output format.

In [98]:
prompt_iterative1= f""" Your task is to organize given information in table format by keeping numarical values. 

### Context:
{text}
###
"""

In [ ]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative1}
    ]
)

print(response.choices[0].message.content)

**Issue 2:** It is long so I need a brief summary.

In [100]:
prompt_iterative2= f""" 
Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation by using at most 20 words.

### Context:
{text}
###
"""

In [ ]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative2}
    ]
)

print(response.choices[0].message.content)

In [102]:
prompt_iterative3= f""" Your task is to organize given information briefly in table format by keeping numarical values.

Then, provide simple explanation of the given context by using at most 20 words.

Format everything as HTML that can be used in a website. 

### Context:
{text}
###
"""

In [ ]:
system_message = "You are a helpful assistant."

response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt_iterative3}
    ]
)

print(response.choices[0].message.content)

## Display the HTML content of the completion response

In [ ]:
from IPython.display import display, HTML
display(HTML(response.choices[0].message.content))

## Self-Ask Prompting  
  
**Self-Ask** is an advanced prompting strategy where the model decomposes a complex question into follow-up sub-questions, answers each one, and then combines the answers for a final response. This is particularly effective for multi-step reasoning tasks.  
  
**Example:**    
*Question: Who lived longer, Theodor Haecker or Harry Vaughan Watkins?*  
  
Here, the model first checks if follow-up questions are needed, generates those sub-questions, provides intermediate answers, then concludes with the final answer.  
  
**Prompt:**  

In [ ]:
# Define the self-ask prompt  
prompt_self_ask = """Question: Who lived longer, Theodor Haecker or Harry Vaughan Watkins?  
  
Are follow up questions needed here? Yes.  
  
Follow up: How old was Theodor Haecker when he died?  
Intermediate answer: Theodor Haecker was 65 years old when he died.  
  
Follow up: How old was Harry Vaughan Watkins when he died?  
Intermediate answer: Harry Vaughan Watkins was 69 years old when he died.  
  
So the final answer is:  
"""  
  
system_message = "You are a helpful assistant."  
response = client.chat.completions.create(  
    model="gpt-4o",  # Replace with your deployment/model name if different  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": prompt_self_ask}  
    ]  
)  
print(response.choices[0].message.content)  

Harry Vaughan Watkins lived longer than Theodor Haecker.


## Self-reflection Prompting  
  
Self-reflection is a prompting technique that guides the model to review, validate, and potentially revise its own output after answering a question. This helps ensure the answer truly meets the requirements—even if the original output was mistaken or incomplete.  
  
Here is a sample sequence using the self-reflection approach:  
  
---  
  
### **Prompt 1: (Initial Reasoning with Chain-of-Thought)**  
  

In [ ]:
# Step 1: Initial prompt asking for a step-by-step solution  
  
prompt_self_reflection_1 = """Suppose I have a cabbage, a goat and a lion, and I need to get them across a river. I have a boat that can only carry myself and a single other item. I am not allowed to leave the cabbage and lion alone together, and I am not allowed to leave the lion and goat alone together.  
  
How can I safely get all three across?  
Please pay attention to the details of the question and think step by step."""  
  
system_message = "You are a helpful assistant."  
response_1 = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": prompt_self_reflection_1}  
    ]  
)  
answer_1 = response_1.choices[0].message.content  
print("Step 1 - Initial Answer:\n", answer_1)  

  
---  
  
### **Prompt 2: (Prompting for Self-reflection)**  
  

In [ ]:
# Step 2: Reflection on the model's prior answer  
  
prompt_self_reflection_2 = "Does the solution meet the assignment? If so, why? If not, why?"  
  
response_2 = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": prompt_self_reflection_1},  
        {"role": "assistant", "content": answer_1},  
        {"role": "user", "content": prompt_self_reflection_2}  
    ]  
)  
answer_2 = response_2.choices[0].message.content  
print("\nStep 2 - Reflection and (if needed) Correction:\n", answer_2)  

## Chain of Verification (CoVe)  
  
Chain of Verification (CoVe) is a prompting method that guides a model to generate verification questions based on its original answers, answer those questions using evidence or external context, and finally use verified facts to correct or refine its initial answer.  
  
---  
  
### **Step 1: Baseline Prompt**  
  
Ask the model to answer the question concisely:  
  
---  
  
### **Step 2: Plan Verification Prompt**  
  
Ask the model to create verification questions based on the baseline answer:  
  
---  
  
### **Step 3: Execute Verification Prompt**  
  
Provide external context (search result) & have the model answer the verification questions:  
  
---  
  
### **Step 4: Final Prompt—Corrected Answer**  
  
Refine or correct the original answer based on verification:  

In [ ]:
# Step 1: Get the baseline answer  
baseline_question = "What was the primary cause of Mexican-American war?"  
system_message = "You are a helpful assistant. Only answer what the question asked, concisely."  
response_baseline = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": baseline_question}  
    ]  
)  
baseline_answer = response_baseline.choices[0].message.content  
print("Baseline Answer:\n", baseline_answer)  
  
# Step 2: Generate verification questions  
plan_verification_prompt = f"""Your task is to create verification questions based on the original question and the following baseline answer. The verification questions are for checking the factual accuracy.  
  
Original Question: {baseline_question}  
Baseline Answer: {baseline_answer}  
  
Please list the verification questions needed to check all major facts in the baseline answer."""  
  
response_verification = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": plan_verification_prompt}  
    ]  
)  
verification_questions = response_verification.choices[0].message.content  
print("\nVerification Questions:\n", verification_questions)  
  
# Step 3: Execute each verification question with context (simulate external search results)  
search_context = """  
[1] 1846-1848  
[2] 1845  
[3] 1836  
"""  
  
execute_verification_prompt = f"""Answer the following questions correctly based on the provided context. Think step by step and provide verified answers.  
  
Questions:  
{verification_questions}  
  
Context: {search_context}  
"""  
  
response_execute = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": execute_verification_prompt}  
    ]  
)  
verified_answers = response_execute.choices[0].message.content  
print("\nVerified Answers:\n", verified_answers)  
  
# Step 4: Refine the baseline answer  
final_prompt = f"""Based on the verified answers below, revise the original answer to include only facts verified by context. Remove any unsupported or incorrect details.  
  
Original Answer:  
{baseline_answer}  
  
Verified Facts:  
{verified_answers}  
"""  
  
response_final = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": system_message},  
        {"role": "user", "content": final_prompt}  
    ]  
)  
final_answer = response_final.choices[0].message.content  
print("\nFinal, Corrected Answer:\n", final_answer)  

## Chain of Density (CoD)  
  
Chain of Density (CoD) is an advanced summarization technique that generates increasingly concise, entity-dense summaries of an article in several iterations. At each step:  
1. The model identifies 1–3 informative entities missing from the previous summary.  
2. The model rewrites the summary to include all prior and new entities—resulting in a denser text of equal length.  
  
Guidelines:  
- The first summary should be long (~80 words), using general language.  
- Each subsequent summary rewrites the prior to improve brevity and information density.  
- Each iteration adds the newly identified "missing entities" to the summary.  
- Never remove entities from previous summaries; always include them in subsequent versions.  

In [ ]:
import json  
import re  
  
# System prompt for Chain of Density  
cod_system_prompt = """  
You will generate increasingly concise, entity-dense summaries of an article given to you.  
Repeat the following two steps 5 times:  
  
Step 1. Identify 1-3 informative entities (", " delimited) from the article which are missing from the previously generated summary.  
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.  
  
A missing entity is:  
- relevant to the main story,  
- specific yet concise (5 words or fewer),  
- novel (not in the previous summary),  
- faithful (present in the article),  
- anywhere (can be located anywhere in the article).  
  
Guidelines:  
- The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly generic language and fillers (e.g., "this article discusses") to reach ~80 words.  
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.  
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".  
- The summaries should become highly concise yet self-contained, i.e., easily understood without the article.  
- Missing entities can appear anywhere in the summary.  
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.  
- Use the exact same number of words for each summary.  
  
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are 'Missing_Entities' and 'Denser_Summary'.  
"""  
  
article = """  
This article discusses an incident that occurred during the Chinese Grand Prix involving two racing drivers, Jenson Button and Pastor Maldonado. The two were competing for the 13th place when Button collided with Maldonado's vehicle, causing damage to both cars. The incident resulted in a penalty for Button, who was demoted to 14th place. Maldonado, on the other hand, had to retire from the race due to the damage his car sustained. During the race, Jenson Button was driving for McLaren, and Pastor Maldonado for Lotus. Fernando Alonso witnessed the collision and managed to avoid it, advancing two places. Button received a five-second penalty and two penalty points on his super license. The incident caused front wing damage to Button's car and rear-end damage to Maldonado's. Nico Rosberg's Mercedes and Alonso's Ferrari were also involved; Sebastian Vettel and Kimi Raikkonen lapped Button during the aftermath. The crash happened on lap 49 of the incident-packed race.  
"""  
  
# Prepare the user message for the model  
initial_user_message = f"""Article:  
{article}  
  
Please start the Chain of Density summarization process as described in the system prompt. Output as JSON."""  
  
# Get the model's output  
response_cod = client.chat.completions.create(  
    model="gpt-4o",  
    messages=[  
        {"role": "system", "content": cod_system_prompt},  
        {"role": "user", "content": initial_user_message}  
    ]  
)  
  
# Extract the raw output and clean up from any extra markdown/text  
raw_out = response_cod.choices[0].message.content.strip()  
  
def extract_json(raw_string):  
    """  
    Extract a JSON array/dict from possibly noisy string using regex.  
    """  
    # Look for the first '[' and the last ']', for a JSON array  
    match = re.search(r'(\[.*\])', raw_string, re.DOTALL)  
    if match:  
        return match.group(1)  
    # Look for curly braces for a dict  
    match = re.search(r'(\{.*\})', raw_string, re.DOTALL)  
    if match:  
        return match.group(1)  
    # Fallback: return raw string  
    return raw_string  
  
# Try parsing the cleaned output as JSON  
try:  
    cod_json = json.loads(extract_json(raw_out))  
    print(json.dumps(cod_json, indent=2))  
except json.JSONDecodeError:  
    print("❗️Model output could not be converted to JSON. Here is the raw text for debugging:")  
    print(raw_out)  

## Self-Consistency (Majority Voting)  
  
**Enhancing Chain of Thought with Multiple Reasoning Paths**  
  
Self-Consistency (Majority Voting) improves reliability for complex reasoning by:  
1. **Sampling the language model multiple times with chain-of-thought prompting** (not just using the first or "greedy" result).  
2. **Aggregating the most consistent numeric answer among all outputs.**  
  
**Steps:**  
1. Prompt the language model with a reasoning question using chain-of-thought.  
2. Run the model several times (e.g., 5-10) to get diverse reasoning paths and answers.  
3. Select the most frequent final answer ("majority vote") as the self-consistent solution.  

In [ ]:
import re  
from collections import Counter  
  
# Chain-of-thought prompt  
prompt = """Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder for $2 per egg. How much does she make every day?  
A:"""  
  
system_message = "You are a helpful assistant. Answer showing step-by-step reasoning."  
  
def get_final_numeric_answer(text):  
    """  
    Extract the final '$' answer from the model's output.  
    """  
    match = re.search(r'The answer is \$?(\d+)\$?', text)  
    if match:  
        return match.group(1)  
    # Fallback: look for $X (e.g. $18)  
    match = re.search(r'\$(\d+)', text)  
    if match:  
        return match.group(1)  
    return None  
  
# Sample the model N times to get different reasoning paths  
answers = []  
n_samples = 8  # Try a few samples; you can increase for more robust voting  
  
for _ in range(n_samples):  
    response = client.chat.completions.create(  
        model="gpt-4o",  
        messages=[  
            {"role": "system", "content": system_message},  
            {"role": "user", "content": prompt}  
        ],  
        temperature=0.8  # Higher temp => more diverse paths  
    )  
    output = response.choices[0].message.content.strip()  
    final = get_final_numeric_answer(output)  
    answers.append(final)  
    print(f"Chain-of-thought Output:\n{output}\n")  
  
# Majority voting (self-consistency)  
filtered = [a for a in answers if a is not None]  
freq = Counter(filtered)  
most_common, freq_count = freq.most_common(1)[0]  
  
print("="*40)  
print(f"Answers: {answers}")  
print(f"Self-Consistent (majority-vote) answer: ${most_common} ({freq_count}/{n_samples} times)")  

# FLARE: Overview  
  
**FLARE (Forward-Looking Active Retrieval Augmented Generation)** is a process where a language model generates text step-by-step, and at each uncertain or knowledge-intensive point, it issues a search query, retrieves external evidence, and incorporates that information before continuing. This greatly reduces hallucinations and improves factuality.  
  
---  
  
## Example: Satya Nadella Biography Summary  
  
### FLARE Steps  
  
**1. Task Definition:**    
Generate a fact-rich summary about Satya Nadella.  
  
**2. Initial Generation:**    
Model starts:    
_"Satya Nadella is a prominent technology executive known for leading Microsoft."_  
  
**3. Forward-Looking Next Phrase Prediction:**    
The model predicts:    
_"He completed his undergraduate studies in..."_  
  
**4. Active Retrieval:**    
Use "**Satya Nadella education**" as a search query.  
  
**5. Retrieval Results:**    
- Bachelor's in Electrical Engineering from Manipal Institute of Technology  
- Master's in Computer Science from University of Wisconsin–Milwaukee  
- MBA from University of Chicago Booth School of Business  
  
**6. Augment Generation:**    
_Incorporate facts:_    
_"He earned a bachelor's degree in electrical engineering from Manipal Institute of Technology, his master’s from the University of Wisconsin–Milwaukee, and an MBA from the University of Chicago Booth School of Business."_  
  
**7. Continue Iterating:**    
Next phrase: "_Before becoming CEO of Microsoft, Nadella..._"    
Query: "**Satya Nadella Microsoft career**", retrieve new facts, augment, and repeat until the summary is complete.  
  
---  
  
## Key FLARE Benefits  
  
- Reduces hallucinated or outdated facts  
- Forces "just-in-time" knowledge integration at generation-time  
- Can be applied to any LLM with a retrieval interface  
  

In [ ]:
import time  
  
# --- Simulated retrieval function ---  
def search(query):  
    """Return simulated search results for given query."""  
    knowledge_base = {  
        "Satya Nadella education": [  
            "Bachelor's in Electrical Engineering from Manipal Institute of Technology",  
            "Master's in Computer Science from University of Wisconsin–Milwaukee",  
            "MBA from University of Chicago Booth School of Business"  
        ],  
        "Satya Nadella Microsoft career": [  
            "Started at Microsoft in 1992",  
            "Led cloud and enterprise group",  
            "Became CEO of Microsoft in 2014"  
        ]  
    }  
    print(f"[Searching: '{query}'] ...")  
    time.sleep(1)  # Simulate search time  
    return knowledge_base.get(query, ["No results found"])  
  
# --- FLARE-style generation steps ---  
summary = "Satya Nadella is a prominent technology executive known for leading Microsoft."  
  
# Step 1: Education  
next_phrase = "He completed his undergraduate studies in..."  
search_results = search("Satya Nadella education")  
education_facts = " ".join(search_results)  
summary += f" {education_facts}"  
  
# Step 2: Microsoft Career  
next_phrase = "Before becoming CEO of Microsoft, Nadella..."  
search_results = search("Satya Nadella Microsoft career")  
career_facts = " ".join(search_results)  
summary += f" {career_facts}"  
  
print("\nFLARE-augmented Summary:\n")  
print(summary)  